In [3]:

import warnings
warnings.filterwarnings('ignore')
import os
import seaborn as sns
import snowflake.connector
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
# import lightgbm and xgboost 
import lightgbm as lgb 
from lightgbm import LGBMRegressor
import xgboost as xgb 
from xgboost import plot_importance
from matplotlib import pyplot

from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score

#RMSE/MAE computation using sklearn library
from sklearn.metrics import mean_squared_error, mean_absolute_error
import scipy
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import ipywidgets as widgets
from IPython.display import display, clear_output

In [4]:

create   table scratch.riders.historical_Utilsation_AOD_daily as
with base as
(
select 
distinct 
lower(city_name) as city_name
,case when zone_code in
('ABAR','AKW','AMR','AMZ','ANH','AQ','AQPP','AQS','ATW','AWH','DBB','DCH','DD11','DDO','DEIR','DFC','DIC','DMEY','DSO',
'DSW','DT','DWS','FCOW','HCC','HPDT','JMH','KAR','MRDF','NAS','RAK','UMS','WAA','ZBL') then  'Deira and Downtown Cluster'
when zone_code in (
'AR','DBRS','DDH','DEGC','DHS','DHS2','DICP','DIP','DJP','DM','DSB','DTS','HPJLT','HPMC','IBN','JBLI','JGE','JI',
'JLT','JVC','JVT','MDW8','PC','PR','RMRM','SC','SPR','SS','TM','TWM') then 'Sportcity and Marina Cluster'
else city_name end as Cluster_Assigned
,Date(START_OF_PERIOD_LOCAL) as dt
-- ,CAST(EXTRACT(HOUR FROM TO_TIMESTAMP(START_OF_PERIOD_LOCAL )) AS INT)as order_created_hour
,case 
     when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (12, 13) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Mon', 'Tue', 'Wed','Thu','Fri') then 'Peak_lunch'
     when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (19, 20) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Mon', 'Tue', 'Wed','Thu') then 'Peak Dinner'
     when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (19, 20) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Fri','Sat','Sun') then 'Super Peak Dinner'
     else 'Other' end as shift

,coalesce(sum(ORDERS_DELIVERED),0)as orders_delievered

,COALESCE(SUM(RET_MINS_SUM ), 0) / NULLIF(COALESCE(SUM(ret_mins_cnt ), 0), 0) AS RET_AVG
,COALESCE(SUM(aod_mins_sum ), 0) / NULLIF(COALESCE(SUM(aod_mins_cnt ), 0), 0) AS AOD_AVG
,NULLIFZERO(sum(ORDERS_ASAP))/NULLIFZERO(sum(RIDERS_AVAILABLE_IN_HOUR)) as TP_1

,NULLIFZERO(sum(RIDER_HOURS_ON_ORDERS_DHW_SUM))/ NULLIFZERO(sum(RIDER_HOURS_WORKED)) as Utilisation_rate_1
,coalesce(sum(TOTAL_ORDER_TIME_PHYSICAL_ZONE_HOURS),0)/NULLIF(COALESCE(sum(TOTAL_TIME_PHYSICAL_ZONE_HOURS ),0),0) as UR_zone_corrected
from
PRODUCTION.AGGREGATE.AGG_ZONE_DELIVERY_METRICS_HOURLY
where  date (start_of_period_local) between '2023-09-01' and '2024-05-30'
-- date (start_of_period_local) between 
--     dateadd('month', -6, date_trunc('month',  current_date()))
--     and  LAST_DAY(dateadd('month', -1, date_trunc('month',  current_date())), MONTH)
and CNT_ERAT >0 -- use this filter to get only active zone
and case when UPPER(country_name) ='KUWAIT' and date (start_of_period_local) between '2024-03-10' and '2024-04-10' then true else false end  =false
group by 1,2,3,4
)
,TP_calc as
(
select 
*
,coalesce(UR_zone_corrected*(60/ RET_avg),TP_1)  as TP_zone_corrected
from 
base 
)
select *exclude(TP_1) from TP_calc;

select count(*), count(distinct CLUSTER_ASSIGNED, dt,shift)  from  scratch.riders.historical_Utilsation_AOD_daily-- 6556 
GRANT SELECT ON TABLE scratch.riders.historical_Utilsation_AOD_daily TO ROLE BI_DEVELOPMENT;





SyntaxError: invalid syntax (675133489.py, line 1)

In [5]:
pd_query="""
select * from scratch.riders.historical_Utilsation_AOD_daily 
"""

conn = snowflake.connector.connect(
        user='Richa.kumari@deliveroo.com',
        password=os.getenv('password'),
        account="deliveroo.eu-central-1",
        authenticator='externalbrowser',
        warehouse="BI_DEVELOPMENT",
        role="BI_DEVELOPMENT",
    )


cur = conn.cursor()
cur.execute(pd_query)
df = cur.fetch_pandas_all()


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://accounts.google.com/o/saml2/idp?idpid=C01jnk96c&SAMLRequest=nZJNb%2BIwEIb%2FSuQ9J3bCh4pFQFkou1S0IKBdLTfjDKmLY2dtpyn%2Ffk0oUvfQHvYQKXKeGT%2BTd4bjt1IGr2Cs0CpFcURQAIrrXKgiRY%2FbWXiDAuuYypnUClJ0AovGo6FlpaxoVrtntYY%2FNVgX%2BEbK0vZDimqjqGZWWKpYCZY6TjfZ%2FYImEaGV0U5zLdGHkq8rmLVgnDe8luRWeL1n5yqKcdM0UdOJtClwQgjBZIA9dUa%2BXfk3P9MnfIxJ98x7wuOrd7fvQl1%2BwVda%2Bwtk6c%2FtdhWulpstCrKr6kQrW5dgNmBeBYfH9eIiYL3B9HYxf7pdL5cR1CEH5QyTYRxZpZuDZEfguqxq51tH%2Fg0fIMdSF8JPP5%2BmqDqK%2FHS%2F3%2BmX5sfNbNo9Lpu79eH3r7uktyvMad%2FEGZ%2FsN5wNZqdyknEUPF3jTc7xzq2tYa7OoTp%2FRJJuSPph0t3GhHY6tNeJev3%2BDgVTH6pQzLWVV3PGua6Vs1GhdSGh9dP4nGCCRV6N%2FSPydELiF3Uc9Dm6bApt7zSj%2F5p%2FiD%2B2eN%2B8Bx%2FGfLrSUvBTMNOmZO7zrOIobk9EHh5alELJhMzy3IC1PjMpdTMxwJxfcGdqQHh0ufXfFR%2F9BQ%3D%3D&Re

In [6]:
print(df.shape)
df.head(2)

df['id']=df['CLUSTER_ASSIGNED']+df['SHIFT'].astype(str)+df['DT'].astype(str)
df['DT']=pd.to_datetime(df['DT'], format='%d%b%Y:%H:%M:%S.%f')
df.head(2)
base_df=df.copy()

(6556, 10)


In [7]:
#  drop the rows with null AOD and UR
base_df=base_df[base_df['UR_ZONE_CORRECTED'].notna()]
print(base_df.shape)
base_df=base_df[base_df['AOD_AVG'].notna()]
print(base_df.shape)
null_df= ((base_df.isna().sum()/ len(base_df))*100).sort_values(ascending=False)
null_df

(6555, 11)
(6552, 11)


UTILISATION_RATE_1    0.015263
CITY_NAME             0.000000
CLUSTER_ASSIGNED      0.000000
DT                    0.000000
SHIFT                 0.000000
ORDERS_DELIEVERED     0.000000
RET_AVG               0.000000
AOD_AVG               0.000000
UR_ZONE_CORRECTED     0.000000
TP_ZONE_CORRECTED     0.000000
id                    0.000000
dtype: float64

In [8]:
base_df.head(1)

,CITY_NAME,CLUSTER_ASSIGNED,DT,SHIFT,ORDERS_DELIEVERED,RET_AVG,AOD_AVG,UTILISATION_RATE_1,UR_ZONE_CORRECTED,TP_ZONE_CORRECTED,id
0,dubai,Sportcity and Marina Cluster,2023-12-19,Peak Dinner,4999,26.539325,36.0851,0.728891,0.744088,1.68223,Sportcity and Marina ClusterPeak Dinner2023-12-19


In [9]:
#  percent
base_df.CLUSTER_ASSIGNED.unique()

array(['Sportcity and Marina Cluster', 'Abu Dhabi', 'Kuwait', 'Ajman',
       'Sharjah', 'Doha', 'Ras Al Khaimah', 'Al Ain',
       'Deira and Downtown Cluster'], dtype=object)

# # Linear Regression Results for UR prediction


In [10]:
df = base_df.copy()
def cap_values(series):
    lower_cap = series.quantile(0.05)
    upper_cap = series.quantile(0.95)
    return np.clip(series, lower_cap, upper_cap)

def calculate_prediction_percentage_errors(y_true, y_pred):
    overpredictions = np.sum(y_pred >= y_true)
    underpredictions = np.sum(y_pred < y_true)
    total_predictions = len(y_true)
    overprediction_pct = (overpredictions / total_predictions) * 100
    underprediction_pct = (underpredictions / total_predictions) * 100
    return overprediction_pct, underprediction_pct


def build_and_evaluate_model(df):
    clusters = df['CLUSTER_ASSIGNED'].unique()
    
    results = []
    df['Predictions_linear_regression'] = np.nan  # Add a column for predictions
    
    for cluster in clusters:
        cluster_df = df[df['CLUSTER_ASSIGNED'] == cluster].copy()
        
        # Cap the values
        cluster_df['AOD_AVG'] = cap_values(cluster_df['AOD_AVG'])
        cluster_df['UR_ZONE_CORRECTED'] = cap_values(cluster_df['UR_ZONE_CORRECTED'])
        
        # Prepare data for linear regression
        X = cluster_df[['AOD_AVG']]
        y = cluster_df['UR_ZONE_CORRECTED']
        
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        # Train the model
        model = LinearRegression()
        model.fit(X, y)
        
        # Make predictions
        predictions = model.predict(X)
        df.loc[df['CLUSTER_ASSIGNED'] == cluster, 'Predictions_linear_regression'] = predictions
        
        # Calculate evaluation metrics
        mae = mean_absolute_error(y, predictions)
        mape = mean_absolute_percentage_error(y, predictions)
        # print((eval_throughput(y,predictions)))
        r2 = r2_score(y, predictions)
        rmse = np.sqrt(mean_squared_error(y, predictions))
        overprediction_pct, underprediction_pct = calculate_prediction_percentage_errors(y, predictions)
        
        

        # Get the regression equation
        intercept = model.intercept_
        coefficient = model.coef_[0]
        equation = f"UR_ZONE_CORRECTED = {intercept:.4f} + {coefficient:.4f} * AOD_AVG"
        
        results.append({
            'CLUSTER_ASSIGNED': cluster,
            'Equation': equation,
            'MAE': mae,
            'MAPE': mape,
            'R2': r2,
            'RMSE': rmse,
            'Overprediction %': overprediction_pct,
            'Underprediction %': underprediction_pct
            
        })
    
    return pd.DataFrame(results), df

linear_results_df, linear_df_with_predictions = build_and_evaluate_model(df)

linear_results_df

,CLUSTER_ASSIGNED,Equation,MAE,MAPE,R2,RMSE,Overprediction %,Underprediction %
0,Sportcity and Marina Cluster,UR_ZONE_CORRECTED = -0.7073 + 0.0379 * AOD_AVG,0.041531,0.063170,0.770216,0.052446,51.285521,48.714479
1,Abu Dhabi,UR_ZONE_CORRECTED = -0.8693 + 0.0407 * AOD_AVG,0.050433,0.080118,0.561048,0.063428,47.225981,52.774019
2,Kuwait,UR_ZONE_CORRECTED = -0.5527 + 0.0323 * AOD_AVG,0.040501,0.069032,0.763835,0.052642,43.865031,56.134969
3,Ajman,UR_ZONE_CORRECTED = -0.2662 + 0.0222 * AOD_AVG,0.069426,0.169093,0.249553,0.086908,51.560380,48.439620
4,Sharjah,UR_ZONE_CORRECTED = -0.4706 + 0.0310 * AOD_AVG,0.057307,0.109250,0.430958,0.070065,52.108844,47.891156
5,Doha,UR_ZONE_CORRECTED = -0.6321 + 0.0299 * AOD_AVG,0.070243,0.124039,0.472712,0.087701,48.849797,51.150203
6,Ras Al Khaimah,UR_ZONE_CORRECTED = -0.5426 + 0.0326 * AOD_AVG,0.074335,0.159247,0.423301,0.092680,50.948509,49.051491
7,Al Ain,UR_ZONE_CORRECTED = -0.2088 + 0.0192 * AOD_AVG,0.072618,0.161884,0.368198,0.094670,46.866485,53.133515
8,Deira and Downtown Cluster,UR_ZONE_CORRECTED = -0.6930 + 0.0354 * AOD_AVG,0.035282,0.055849,0.660259,0.044328,46.278755,53.721245


In [11]:
linear_df_with_predictions.head(1)

,CITY_NAME,CLUSTER_ASSIGNED,DT,SHIFT,ORDERS_DELIEVERED,RET_AVG,AOD_AVG,UTILISATION_RATE_1,UR_ZONE_CORRECTED,TP_ZONE_CORRECTED,id,Predictions_linear_regression
0,dubai,Sportcity and Marina Cluster,2023-12-19,Peak Dinner,4999,26.539325,36.0851,0.728891,0.744088,1.68223,Sportcity and Marina ClusterPeak Dinner2023-12-19,0.660458


In [12]:
linear_df_with_predictions.groupby(['CLUSTER_ASSIGNED','SHIFT'])[['UR_ZONE_CORRECTED','Predictions_linear_regression']].mean()


UR_ZONE_CORRECTED  \
CLUSTER_ASSIGNED             SHIFT                                  
Abu Dhabi                    Other                       0.592476   
                             Peak Dinner                 0.738467   
                             Peak_lunch                  0.605541   
                             Super Peak Dinner           0.718588   
Ajman                        Other                       0.434637   
                             Peak Dinner                 0.473260   
                             Peak_lunch                  0.409611   
                             Super Peak Dinner           0.485669   
Al Ain                       Other                       0.488361   
                             Peak Dinner                 0.575351   
                             Peak_lunch                  0.398643   
                             Super Peak Dinner           0.553850   
Deira and Downtown Cluster   Other                       0.604452   
                             Peak Dinner                 0.672245   
                             Peak_lunch                  0.679625   
                             Super Peak Dinner           0.686288   
Doha                         Other                       0.580465   
                             Peak Dinner                 0.649758   
                             Peak_lunch                  0.604271   
                             Super Peak Dinner           0.636379   
Kuwait                       Other                       0.615679   
                             Peak Dinner                 0.727153   
                             Peak_lunch                  0.522508   
                             Super Peak Dinner           0.727392   
Ras Al Khaimah               Other                       0.477378   
                             Peak Dinner                 0.552319   
                             Peak_lunch                  0.444578   
                             Super Peak Dinner           0.566520   
Sharjah                      Other                       0.518400   
                             Peak Dinner                 0.554257   
                             Peak_lunch                  0.523999   
                             Super Peak Dinner           0.578296   
Sportcity and Marina Cluster Other                       0.580899   
                             Peak Dinner                 0.755459   
                             Peak_lunch                  0.638679   
                             Super Peak Dinner           0.777696   

                                                Predictions_linear_regression  
CLUSTER_ASSIGNED             SHIFT                                             
Abu Dhabi                    Other                                   0.608413  
                             Peak Dinner                             0.685964  
                             Peak_lunch                              0.639647  
                             Super Peak Dinner                       0.698797  
Ajman                        Other                                   0.432788  
                             Peak Dinner                             0.444504  
                             Peak_lunch                              0.440298  
                             Super Peak Dinner                       0.464553  
Al Ain                       Other                                   0.477636  
                             Peak Dinner                             0.513074  
                             Peak_lunch                              0.475159  
                             Super Peak Dinner                       0.531916  
Deira and Downtown Cluster   Other                                   0.609210  
                             Peak Dinner                             0.667056  
                             Peak_lunch                              0.678639  
                             Super Peak Dinner                   

##   Interactive output for the above method

In [13]:

def display_equation_and_predict(cluster):
    clear_output()  # Clear previous output
    
    # Display the regression equation for the selected cluster
    result_row = linear_results_df[linear_results_df['CLUSTER_ASSIGNED'] == cluster].iloc[0]
    equation = result_row['Equation']
    print(f"Equation for Cluster {cluster}: {equation}")
    
    # Create an input box for AOD value
    aod_value = widgets.FloatText(
        value=0,
        description='AOD Value:',
        disabled=False
    )
    
    # Create an output area for displaying the prediction
    output = widgets.Output()
    
    # Function to handle button click and display prediction
    def on_button_clicked(b):
        with output:
            clear_output()  # Clear previous output in the output area
            aod = aod_value.value
            intercept, coefficient = result_row['Equation'].split(' = ')[1].split(' + ')
            coefficient = float(coefficient.split(' * ')[0])
            intercept = float(intercept)
            prediction = intercept + coefficient * aod
            print(f"Predicted UR_ZONE_CORRECTED for AOD {aod}: {prediction:.2f}")
    
    # Create a button for making predictions
    button = widgets.Button(
        description='Predict UR_ZONE_CORRECTED',
        disabled=False,
        button_style='',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click to make prediction',
        icon='check'
    )
    
    button.on_click(on_button_clicked)
    
    # Display the input box, button, and output area
    display(aod_value, button, output)

# Create a dropdown for selecting cluster
cluster_dropdown = widgets.Dropdown(
    options=linear_results_df['CLUSTER_ASSIGNED'].unique(),
    value=linear_results_df['CLUSTER_ASSIGNED'].unique()[0],
    description='Cluster:',
    disabled=False
)

# Display interactive widgets
widgets.interactive(display_equation_and_predict, cluster=cluster_dropdown)


interactive(children=(Dropdown(description='Cluster:', options=('Sportcity and Marina Cluster', 'Abu Dhabi', '…

##  Use Xgb model to predict the output


In [14]:

df = base_df.copy()

# Parameters for XGBoost
params = {
    'objective': 'reg:squarederror',
    'max_depth': 3,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'random_state': 42
}

def cap_values(series):
    lower_cap = series.quantile(0.05)
    upper_cap = series.quantile(0.98)
    return np.clip(series, lower_cap, upper_cap)

def calculate_prediction_percentage_errors(y_true, y_pred):
    overpredictions = np.sum(y_pred > y_true)
    underpredictions = np.sum(y_pred < y_true)
    total_predictions = len(y_true)
    overprediction_pct = (overpredictions / total_predictions) * 100
    underprediction_pct = (underpredictions / total_predictions) * 100
    return overprediction_pct, underprediction_pct

def build_and_evaluate_model(df):
    clusters = df['CLUSTER_ASSIGNED'].unique()
    
    results = []
    df['Predictions_XGB'] = np.nan  # Add a column for predictions
    
    models = {}  # Store the models for each cluster
    
    for cluster in clusters:
        cluster_df = df[df['CLUSTER_ASSIGNED'] == cluster].copy()
        
        # Cap the values
        cluster_df['AOD_AVG'] = cap_values(cluster_df['AOD_AVG'])
        cluster_df['UR_ZONE_CORRECTED'] = cap_values(cluster_df['UR_ZONE_CORRECTED'])
        
        # Prepare data for XGBoost regression
        X = cluster_df[['AOD_AVG']]
        y = cluster_df['UR_ZONE_CORRECTED']
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
        
        # Train the XGBoost model
        model_XGB = xgb.XGBRegressor(**params)
        model_XGB.fit(x_train, y_train)
        
        # Save the model
        models[cluster] = model_XGB
        
        # Make predictions
        predictions = model_XGB.predict(X)
        df.loc[df['CLUSTER_ASSIGNED'] == cluster, 'Predictions_XGB'] = predictions
        
        # Calculate evaluation metrics
        mae = mean_absolute_error(y, predictions)
        mape = mean_absolute_percentage_error(y, predictions)
        r2 = r2_score(y, predictions)
        rmse = np.sqrt(mean_squared_error(y, predictions))
        overprediction_pct, underprediction_pct = calculate_prediction_percentage_errors(y, predictions)
        
        results.append({
            'CLUSTER_ASSIGNED': cluster,
            'MAE': mae,
            'MAPE': mape,
            'R2': r2,
            'RMSE': rmse,
            'Overprediction %': overprediction_pct,
            'Underprediction %': underprediction_pct
        })
    
    return pd.DataFrame(results), df, models

xgb_results_df, xgb_df_with_predictions, models = build_and_evaluate_model(df)
xgb_results_df

,CLUSTER_ASSIGNED,MAE,MAPE,R2,RMSE,Overprediction %,Underprediction %
0,Sportcity and Marina Cluster,0.033535,0.051702,0.851614,0.043360,48.443843,51.556157
1,Abu Dhabi,0.047554,0.075093,0.615912,0.060453,49.391069,50.608931
2,Kuwait,0.037157,0.062994,0.805002,0.049178,47.699387,52.300613
3,Ajman,0.066889,0.159947,0.359227,0.085791,49.525102,50.474898
4,Sharjah,0.054748,0.103454,0.513865,0.068671,52.789116,47.210884
5,Doha,0.063802,0.112190,0.568922,0.081980,49.932341,50.067659
6,Ras Al Khaimah,0.069851,0.146503,0.521812,0.087980,49.864499,50.135501
7,Al Ain,0.068019,0.150137,0.478542,0.088043,47.547684,52.452316
8,Deira and Downtown Cluster,0.033333,0.052167,0.702588,0.043188,48.173207,51.826793


In [15]:
xgb_df_with_predictions.groupby(['CLUSTER_ASSIGNED','SHIFT'])[['UR_ZONE_CORRECTED','Predictions_XGB']].mean()


UR_ZONE_CORRECTED  \
CLUSTER_ASSIGNED             SHIFT                                  
Abu Dhabi                    Other                       0.592476   
                             Peak Dinner                 0.738467   
                             Peak_lunch                  0.605541   
                             Super Peak Dinner           0.718588   
Ajman                        Other                       0.434637   
                             Peak Dinner                 0.473260   
                             Peak_lunch                  0.409611   
                             Super Peak Dinner           0.485669   
Al Ain                       Other                       0.488361   
                             Peak Dinner                 0.575351   
                             Peak_lunch                  0.398643   
                             Super Peak Dinner           0.553850   
Deira and Downtown Cluster   Other                       0.604452   
                             Peak Dinner                 0.672245   
                             Peak_lunch                  0.679625   
                             Super Peak Dinner           0.686288   
Doha                         Other                       0.580465   
                             Peak Dinner                 0.649758   
                             Peak_lunch                  0.604271   
                             Super Peak Dinner           0.636379   
Kuwait                       Other                       0.615679   
                             Peak Dinner                 0.727153   
                             Peak_lunch                  0.522508   
                             Super Peak Dinner           0.727392   
Ras Al Khaimah               Other                       0.477378   
                             Peak Dinner                 0.552319   
                             Peak_lunch                  0.444578   
                             Super Peak Dinner           0.566520   
Sharjah                      Other                       0.518400   
                             Peak Dinner                 0.554257   
                             Peak_lunch                  0.523999   
                             Super Peak Dinner           0.578296   
Sportcity and Marina Cluster Other                       0.580899   
                             Peak Dinner                 0.755459   
                             Peak_lunch                  0.638679   
                             Super Peak Dinner           0.777696   

                                                Predictions_XGB  
CLUSTER_ASSIGNED             SHIFT                               
Abu Dhabi                    Other                     0.608731  
                             Peak Dinner               0.690515  
                             Peak_lunch                0.637145  
                             Super Peak Dinner         0.706369  
Ajman                        Other                     0.437106  
                             Peak Dinner               0.452710  
                             Peak_lunch                0.437684  
                             Super Peak Dinner         0.467138  
Al Ain                       Other                     0.477082  
                             Peak Dinner               0.526067  
                             Peak_lunch                0.465874  
                             Super Peak Dinner         0.544396  
Deira and Downtown Cluster   Other                     0.607811  
                             Peak Dinner               0.670311  
                             Peak_lunch                0.682299  
                             Super Peak Dinner         0.681707  
Doha                         Other                     0.590347  
                             Peak Dinner               0.637518  
                             Peak_lunch                0.613675  
                             Super Peak

In [16]:
# Function to display model info and predict UR_ZONE_CORRECTED
def display_info_and_predict(cluster):
    clear_output()  # Clear previous output
    
    # Display the model info for the selected cluster
    result_row = xgb_results_df[xgb_results_df['CLUSTER_ASSIGNED'] == cluster].iloc[0]
    print(f"Model Metrics for Cluster {cluster}:")
    print(f"MAE: {result_row['MAE']:.2f}")
    print(f"MAPE: {result_row['MAPE']:.2f}")
    print(f"R2: {result_row['R2']:.2f}")
    print(f"Overprediction %: {result_row['Overprediction %']:.2f}%")
    print(f"Underprediction %: {result_row['Underprediction %']:.2f}%")
    
    # Create an input box for AOD value
    aod_value = widgets.FloatText(
        value=0,
        description='AOD Value:',
        disabled=False
    )
    
    # Create an output area for displaying the prediction
    output = widgets.Output()
    
    # Function to handle button click and display prediction
    def on_button_clicked(b):
        with output:
            clear_output()  # Clear previous output in the output area
            aod = aod_value.value
            X_new = np.array([[aod]])
            model_XGB = models[cluster]
            prediction = model_XGB.predict(X_new)[0]
            print(f"Predicted UR_ZONE_CORRECTED for AOD {aod}: {prediction:.2f}")
    
    # Create a button for making predictions
    button = widgets.Button(
        description='Predict UR_ZONE_CORRECTED',
        disabled=False,
        button_style='',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click to make prediction',
        icon='check'
    )
    
    button.on_click(on_button_clicked)
    
    # Display the input box, button, and output area
    display(aod_value, button, output)

# Create a dropdown for selecting cluster
cluster_dropdown = widgets.Dropdown(
    options=xgb_results_df['CLUSTER_ASSIGNED'].unique(),
    value=xgb_results_df['CLUSTER_ASSIGNED'].unique()[0],
    description='Cluster:',
    disabled=False
)

# Display interactive widgets
interactive_output = widgets.interactive(display_info_and_predict, cluster=cluster_dropdown)
display(cluster_dropdown, interactive_output)


Dropdown(description='Cluster:', options=('Sportcity and Marina Cluster', 'Abu Dhabi', 'Kuwait', 'Ajman', 'Sha…

interactive(children=(Dropdown(description='Cluster:', options=('Sportcity and Marina Cluster', 'Abu Dhabi', '…

In [17]:
#  join the 2 output df to show result of both the xgb as well as linear regression model
# Merge DataFrames on the 'ID' column
merged_df = pd.merge(xgb_df_with_predictions[['CLUSTER_ASSIGNED','SHIFT','Predictions_XGB','id','UR_ZONE_CORRECTED','ORDERS_DELIEVERED']], linear_df_with_predictions[['Predictions_linear_regression','id']]
                     , on='id')
merged_df.head(2)

,CLUSTER_ASSIGNED,SHIFT,Predictions_XGB,id,UR_ZONE_CORRECTED,ORDERS_DELIEVERED,Predictions_linear_regression
0,Sportcity and Marina Cluster,Peak Dinner,0.705885,Sportcity and Marina ClusterPeak Dinner2023-12-19,0.744088,4999,0.660458
1,Abu Dhabi,Peak_lunch,0.657038,Abu DhabiPeak_lunch2023-11-09,0.645813,1359,0.656811


In [18]:
merged_df.groupby(['CLUSTER_ASSIGNED','SHIFT'])[['UR_ZONE_CORRECTED','Predictions_XGB','Predictions_linear_regression']].mean()
# merged_df.to_csv('merged_df.csv')


# Function to calculate weighted average
def weighted_avg(values, weights):
    return (values * weights).sum() / weights.sum()

# Group by 'Group' and calculate weighted average of 'Value' using 'Weight'
weighted_avg_df = merged_df.groupby(['CLUSTER_ASSIGNED','SHIFT']).apply(lambda x: pd.Series({
    'Weighted_Avg': weighted_avg(x['Predictions_linear_regression'], x['ORDERS_DELIEVERED'])
})).reset_index()


weighted_avg_df.to_csv('weighted_avg_df.csv')
weighted_avg_df

,CLUSTER_ASSIGNED,SHIFT,Weighted_Avg
0,Abu Dhabi,Other,0.610623
1,Abu Dhabi,Peak Dinner,0.692215
2,Abu Dhabi,Peak_lunch,0.646075
3,Abu Dhabi,Super Peak Dinner,0.706938
4,Ajman,Other,0.431998
5,Ajman,Peak Dinner,0.447417
6,Ajman,Peak_lunch,0.437510
7,Ajman,Super Peak Dinner,0.468308
8,Al Ain,Other,0.475162
9,Al Ain,Peak Dinner,0.516581


In [22]:
x=merged_df.groupby(['CLUSTER_ASSIGNED','SHIFT'])[['UR_ZONE_CORRECTED','Predictions_XGB','Predictions_linear_regression']].mean().reset_index()
x.to_csv('x.csv')

In [25]:
linear_results_df.to_csv('eqn.csv')

In [24]:
xgb_results_df

,CLUSTER_ASSIGNED,MAE,MAPE,R2,RMSE,Overprediction %,Underprediction %
0,Sportcity and Marina Cluster,0.033535,0.051702,0.851614,0.043360,48.443843,51.556157
1,Abu Dhabi,0.047554,0.075093,0.615912,0.060453,49.391069,50.608931
2,Kuwait,0.037157,0.062994,0.805002,0.049178,47.699387,52.300613
3,Ajman,0.066889,0.159947,0.359227,0.085791,49.525102,50.474898
4,Sharjah,0.054748,0.103454,0.513865,0.068671,52.789116,47.210884
5,Doha,0.063802,0.112190,0.568922,0.081980,49.932341,50.067659
6,Ras Al Khaimah,0.069851,0.146503,0.521812,0.087980,49.864499,50.135501
7,Al Ain,0.068019,0.150137,0.478542,0.088043,47.547684,52.452316
8,Deira and Downtown Cluster,0.033333,0.052167,0.702588,0.043188,48.173207,51.826793


In [ ]:
Peak dinner
other peak
